# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "mtecbo"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 10:26:25,522 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("CODIGO", T.StringType(), True),
    T.StructField("TITULO", T.StringType(), True)
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("mte/cbo")
local = f"s3a://landing/mte/cbo/{ano_mes}/CBO2002 - Grande Grupo.csv"

In [5]:
config_adicional = {
    'header': True,
    'sep': ';',
    'encoding': 'iso-8859-1'
}

# Leitura do arquivo

In [6]:
df_compliance_cbo_grande_grupo = extrator_bronze.criar_view_temporaria_arquivo(local, 'compliance_cbo_grande_grupo',schema,**config_adicional)

2024-10-15 10:26:29,656 - INFO - Lista de yaml carregada com sucesso
2024-10-15 10:26:32,871 - INFO - Criando view temporaria.


In [7]:
sql_query = """
    SELECT
        CODIGO AS cd,
        '' AS demanda_formacao,
        TITULO AS grande_grupo,
        '' AS subgrupo_principal,
        '' AS subgrupo,
        '' AS familia,
        '' AS ocupacao,
        'GRANDE GRUPO' AS nivel,
        '' AS industrial_uniepro
    FROM compliance_cbo_grande_grupo 
"""

In [8]:
df_compliance_cbo_grande_grupo = extrator_bronze.carregar_dados_delta(sql_query)

2024-10-15 10:26:32,883 - INFO - Aguarde... Executando Query (Delta)
2024-10-15 10:26:32,884 - INFO - 
    SELECT
        CODIGO AS cd,
        '' AS demanda_formacao,
        TITULO AS grande_grupo,
        '' AS subgrupo_principal,
        '' AS subgrupo,
        '' AS familia,
        '' AS ocupacao,
        'GRANDE GRUPO' AS nivel,
        '' AS industrial_uniepro
    FROM compliance_cbo_grande_grupo 

2024-10-15 10:26:33,045 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [9]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/mte/cbograndegrupo'

In [10]:
extrator_bronze.salvar_delta(df_compliance_cbo_grande_grupo, 'overwrite')

2024-10-15 10:26:33,056 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 10:26:44,636 - INFO - Dados persistidos com sucesso
2024-10-15 10:26:44,637 - INFO - s3a://bronze/mte/cbograndegrupo
2024-10-15 10:26:44,638 - INFO - Aguarde... Realizando optimize
2024-10-15 10:26:51,440 - INFO - Optimize executado com sucesso.
2024-10-15 10:26:51,441 - INFO - Aguarde... Realizando vacuum
2024-10-15 10:27:15,933 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [11]:
extrator_bronze.parar_sessao()

2024-10-15 10:27:16,484 - INFO - Sessão Spark finalizada.
